Тема “Перевод без механизма внимания 
SEQ2SEQ модель”

Разобраться с моделькой перевода (без механизма внимания) как она устроена, запустить для перевода с русского на английский (при желании можно взять другие пары языков)

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import numpy as np

import unicodedata
import re
import os
import io
from pathlib import Path

In [2]:
%%capture
!wget http://www.manythings.org/anki/rus-eng.zip
!rm ru_en -r
!mkdir ru_en
!unzip rus-eng.zip -d ru_en
!rm rus-eng.zip

In [3]:
!ls ru_en

_about.txt  rus.txt


In [4]:
data_path = Path('/content/ru_en/rus.txt')

In [5]:
def preprocess_sentence(w):
  w = w.lower().strip()

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [6]:
preprocess_sentence("This is example sentence.")

'<start> this is example sentence . <end>'

In [7]:
def create_dataset(data_path: Path, num_examples: int) -> zip:
  lines = data_path.read_text(encoding='UTF-8').strip().split('\n')

  word_pairs = [[preprocess_sentence(sent) for sent in line.split('\t')[:2]] for line in lines[:num_examples]]

  return zip(*word_pairs)

def load_dataset(data_path: Path, num_examples: int) -> tuple:
  # creating cleaned input, output pairs
  target_lang, input_lang = create_dataset(data_path, num_examples)

  input_tensor, input_lang_tokenizer = tokenize(input_lang)
  target_tensor, target_lang_tokenizer = tokenize(target_lang)

  return input_tensor, target_tensor, input_lang_tokenizer, target_lang_tokenizer

def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

  return tensor, lang_tokenizer

In [8]:
en, ru = create_dataset(data_path, None)
print(f'len(en) = {len(en)}, len(ru) = {len(ru)}')
print(en[0])
print(ru[0])

len(en) = 451436, len(ru) = 451436
<start> go . <end>
<start> марш ! <end>


In [9]:
NUM_EXAMPLES = 100000
input_tensor, target_tensor, input_lang_tokenizer, target_lang_tokenizer = load_dataset(data_path, NUM_EXAMPLES)

In [10]:
# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]
max_length_targ, max_length_inp

(11, 15)

In [11]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

80000 80000 20000 20000


In [12]:
def show_mapping(tokenizer, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, tokenizer.index_word[t]))

print ("Input Language; index to word mapping")
show_mapping(input_lang_tokenizer, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
show_mapping(target_lang_tokenizer, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
4 ----> я
9322 ----> прослежу
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
38 ----> i'll
174 ----> make
263 ----> sure
3 ----> .
2 ----> <end>


In [13]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
STEPS_PER_EPOCH = len(input_tensor_train)//BATCH_SIZE
EMBEDDING_DIM = 300
UNITS = 1024
vocab_inp_size = len(input_lang_tokenizer.word_index)+1
vocab_tar_size = len(target_lang_tokenizer.word_index)+1

In [14]:
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [15]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 15]), TensorShape([64, 11]))

In [16]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units, batch_size, with_attention=False):
        super(Encoder, self).__init__()

        self.batch_size = batch_size
        self.rnn_units = rnn_units

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.rnn_units, return_sequences=with_attention, return_state=True, recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return state

    def get_zero_hidden_state(self):
        return tf.zeros((self.batch_size, self.rnn_units))

In [17]:
encoder = Encoder(vocab_inp_size, EMBEDDING_DIM, UNITS, BATCH_SIZE)

In [18]:
# sample input
sample_hidden = encoder.get_zero_hidden_state()
sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder Hidden state shape: (batch size, units) (64, 1024)


In [19]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units, batch_size):
        super(Decoder, self).__init__()

        self.batch_size = batch_size
        self.rnn_units = rnn_units
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.rnn_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x, hidden):
        # enc_output shape == (batch_size, max_length, hidden_size)
        x = self.embedding(x) # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        output, state = self.gru(x, initial_state=hidden)
        output = tf.reshape(output, (-1, output.shape[2])) # output shape == (batch_size * 1, hidden_size)
        x = self.fc(output) # output shape == (batch_size, vocab)

        return x, state

In [20]:
decoder = Decoder(vocab_tar_size, EMBEDDING_DIM, UNITS, BATCH_SIZE)

In [21]:
decoder_sample_x, decoder_sample_h = decoder(tf.random.uniform((BATCH_SIZE, 1)), sample_hidden)
print(f'{decoder_sample_x.shape}')
print(f'{decoder_sample_h.shape}')

(64, 7335)
(64, 1024)


In [22]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_funcion(real, pred):
    
    loss_ = loss_object(real, pred)

    mask = tf.math.logical_not(tf.math.equal(real, 0))
    mask = tf.cast(mask, dtype=loss_.dtype)

    loss_ *= mask

    return tf.reduce_mean(loss_)

In [23]:
@tf.function
def train_step(input, target, encoder_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        encoder_hidden = encoder(input, encoder_hidden)

        decoder_hidden = encoder_hidden

        decoder_input = tf.expand_dims([target_lang_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

        # Teacher forcing - feeding the target as the next input
        for t in range(1, target.shape[1]):
            # passing enc_output to the decoder
            predictions, decoder_hidden = decoder(decoder_input, decoder_hidden)

            loss += loss_funcion(target[:, t], predictions)

            # using teacher forcing
            decoder_input = tf.expand_dims(target[:, t], 1)

    batch_loss = (loss / int(target.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [24]:
EPOCHS = 50

for epoch in range(EPOCHS):
    
    encoder_hidden = encoder.get_zero_hidden_state()
    total_loss = 0

    for (batch, (input, target)) in enumerate(dataset.take(STEPS_PER_EPOCH)):
      batch_loss = train_step(input, target, encoder_hidden)
      total_loss += batch_loss

      if batch % 100 == 0: print(f'Epoch {epoch+1} Batch {batch} Loss {batch_loss:.4f}')

    print(f'Epoch {epoch+1} Batch {batch} Loss {total_loss / STEPS_PER_EPOCH:.4f}')

Epoch 1 Batch 0 Loss 4.5136
Epoch 1 Batch 100 Loss 2.0369
Epoch 1 Batch 200 Loss 1.9116
Epoch 1 Batch 300 Loss 1.6086
Epoch 1 Batch 400 Loss 1.4424
Epoch 1 Batch 500 Loss 1.3796
Epoch 1 Batch 600 Loss 1.3039
Epoch 1 Batch 700 Loss 1.1726
Epoch 1 Batch 800 Loss 1.1430
Epoch 1 Batch 900 Loss 1.1927
Epoch 1 Batch 1000 Loss 1.0589
Epoch 1 Batch 1100 Loss 0.9558
Epoch 1 Batch 1200 Loss 0.9683
Epoch 1 Batch 1249 Loss 1.4021
Epoch 2 Batch 0 Loss 0.7586
Epoch 2 Batch 100 Loss 0.8149
Epoch 2 Batch 200 Loss 0.7487
Epoch 2 Batch 300 Loss 0.7502
Epoch 2 Batch 400 Loss 0.6348
Epoch 2 Batch 500 Loss 0.6098
Epoch 2 Batch 600 Loss 0.6372
Epoch 2 Batch 700 Loss 0.6161
Epoch 2 Batch 800 Loss 0.6495
Epoch 2 Batch 900 Loss 0.6156
Epoch 2 Batch 1000 Loss 0.5468
Epoch 2 Batch 1100 Loss 0.6064
Epoch 2 Batch 1200 Loss 0.5857
Epoch 2 Batch 1249 Loss 0.6690
Epoch 3 Batch 0 Loss 0.3890
Epoch 3 Batch 100 Loss 0.3527
Epoch 3 Batch 200 Loss 0.3569
Epoch 3 Batch 300 Loss 0.4140
Epoch 3 Batch 400 Loss 0.3277
Epoch 3 

In [25]:
def evaluate(sentence):
  attantion_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  inputs = [input_lang_tokenizer.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, UNITS))]
  enc_hidden = encoder(inputs, hidden)
  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([target_lang_tokenizer.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden = decoder(dec_input, dec_hidden)

    # storing the attention weights to plot later on
    predicted_id = tf.argmax(predictions[0]).numpy()
    result += target_lang_tokenizer.index_word[predicted_id] + ' '

    if target_lang_tokenizer.index_word[predicted_id] == '<end>':
      return result, sentence

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence

In [26]:
def translate(sentence):
  result, sentence = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

Проверка перевода

In [27]:
translate('Здесь хорошо.')

Input: <start> здесь хорошо . <end>
Predicted translation: it's good here . <end> 


In [28]:
translate('я не смогу поехать.')

Input: <start> я не смогу поехать . <end>
Predicted translation: i can't go . <end> 


In [29]:
translate(u'Вы все еще дома?')

Input: <start> вы еще дома ? <end>
Predicted translation: are you still at home ? <end> 
